In [2]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [3]:
class BestmanState(TypedDict):

    run:int
    ball:int
    four:int
    six:int
    
    sr:float
    bpb:float
    boundary_per:float
    summary:str

In [4]:
def strike_rate(state:BestmanState) -> BestmanState:

    runs=state['run']
    balls=state['ball']

    sr = round((runs/balls)*100,2)

    return {'sr':sr}

In [5]:
def b_p_boundary(state:BestmanState) -> BestmanState:

    balls = state['ball']
    four = state['four']
    six = state['six']

    tot = four+six
    bpb = round(balls/tot,2)

    return {'bpb':bpb}


In [6]:
def b_percent(state:BestmanState) -> BestmanState:

    runs = state['run']
    four = state['four']
    six = state['six']

    tot = (four*4)+(six*6)
    b_pr = round((tot/runs)*100,2)
    
    return {'boundary_per':b_pr}

In [7]:
def summary(state:BestmanState) -> BestmanState:

    summary = f"""
Strike Rate - {state['sr']} \n
Balls per boundry - {state['bpb']} \n
Boundry Percentage - {state['boundary_per']}
"""
    
    state['summary'] = summary
    return state

In [8]:
graph = StateGraph(BestmanState)

graph.add_node("strike_rate",strike_rate)
graph.add_node("b_p_boundary",b_p_boundary)
graph.add_node("b_percent",b_percent)
graph.add_node("summary",summary)

graph.add_edge(START,"strike_rate")
graph.add_edge(START,"b_p_boundary")
graph.add_edge(START,"b_percent")

graph.add_edge("strike_rate","summary")
graph.add_edge("b_p_boundary","summary")
graph.add_edge("b_percent","summary")

graph.add_edge("summary",END)

workflow = graph.compile()

In [9]:
initial_state = {
    "run":100,
    "ball":50,
    "four":6,
    "six":4
}

final_state = workflow.invoke(initial_state)

print(final_state['summary'])


Strike Rate - 200.0 

Balls per boundry - 5.0 

Boundry Percentage - 48.0

